In [49]:
import numpy as np
import pandas as pd
import re
import string
import nltk
import pickle

In [50]:
with open('../static/model/model.pickle', 'rb') as f:
    model = pickle.load(f)

In [51]:
def lower_case(text):
    text = text.lower()
    return text

In [52]:
def remove_punctuations(text):
    for punctuation in string.punctuation:
        text = text.replace(punctuation, '')
    return text

In [53]:
with open ('../static/model/corpora/stopwords/english', 'r') as file:
    sw = file.read().splitlines()

In [54]:
from nltk.stem import PorterStemmer
ps = PorterStemmer()

In [55]:
vocab = pd.read_csv('../static/model/vocabulary.txt', header = None)
tokens = vocab[0].tolist()

In [56]:
def text_preprocessing(text):
    data = pd.DataFrame([text], columns=['tweet'])
    data['tweet'] = data['tweet'].apply(lower_case)
    data['tweet'] = data['tweet'].apply(lambda x: " ".join(re.sub(r'^https://.*[\r\n]*', '', x, flags = re.MULTILINE) for x in x.split()))
    data['tweet'] = data['tweet'].apply(remove_punctuations)
    data['tweet'] = data['tweet'].str.replace(r'\d+', '', regex = True)
    data['tweet'] = data['tweet'].apply(lambda x: " ".join(ps.stem(x) for x in x.split()))
    data['tweet'] = data['tweet'].apply(lambda x: " ".join(x for x in x.split() if x not in sw))
    return data['tweet']

In [58]:
def vectorizer(ds, vocabulary):
    vectorized_list = []
    
    for sentence in ds:
        sentence_list = np.zeros(len(vocabulary))

        for i in range(len(vocabulary)):
            if vocabulary[i] in sentence.split():
                sentence_list[i] = 1

        vectorized_list.append(sentence_list)

    vectorized_list_new = np.asarray(vectorized_list, dtype = np.float32)

    return vectorized_list_new

In [59]:
def get_prediction(text):
    preprocessed_text = text_preprocessing(text)
    vectorized_text = vectorizer(preprocessed_text, tokens)
    prediction = model.predict(vectorized_text)
    if prediction == 1:
        return 'negative'
    else:
        return 'positive'

In [61]:
get_prediction("I hate it")

'negative'